Code from here: 
https://pystac.readthedocs.io/en/stable/tutorials/creating-a-landsat-stac.html

Also to run this notebook you’ll need jupyter installed locally as well. If you’re running in a docker container, make sure that port 5555 is exposed if you want to run the server at the end of the notebook.

In [ ]:
from typing import Any, Dict, List, Optional, cast
from typing_extensions import TypedDict

import pystac
from pystac.extensions.eo import EOExtension
from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.view import ViewExtension

AWS keeps a scene list that includes information about where the scene is located and how to access it’s information. Landsat 8 was reorganized into a Collection system in the past, and so there’s two places to read a scene list from (as mentioned on the aws page). We’ll pull from the data that is organized as Collection 1 data, which is where all new processed data is added since 2017.

In [ ]:
import csv
from datetime import datetime
import gzip
from io import StringIO
from urllib.request import urlopen

In [ ]:
# Here we use a TypedDict to define the structure of the Landsat scene dictionaries
# we will be working with. You can read more about the TypedDict class here:
# https://docs.python.org/3/library/typing.html#typing.TypedDict
class Scene(TypedDict):
    productId: str
    entityId: str
    acquisitionDate: str
    cloudCover: str
    processingLevel: str
    path: str
    row: str
    min_lat: str
    min_lon: str
    max_lat: str
    max_lon: str
    download_url: str


# Collection 1 data
url = "https://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz"

# Read and unzip the content
response = urlopen(url)
gunzip_response = gzip.GzipFile(fileobj=response)
content = gunzip_response.read()

# Read the scenes in as dictionaries
scenes: List[Scene] = list(
    map(lambda s: cast(Scene, s), csv.DictReader(StringIO(content.decode("utf-8"))))
)

In [ ]:
len(scenes)

As you can see, there are a lot of scenes! Even though STAC items contain just the metadata for the scenes (and not the bulky raster image), this would still be a lot of data and files to work with for this tutorial.

Let’s see what one of the scenes looks like, and then filter on those properties to scope things down.

In [ ]:
scenes[0]

As you can see, we have both a date and a location that we can filter on.

#Filter scenes by a location
Let’s pick a location to filter the scenes by. Here we choose Philly, but feel free to change the location by modifying the latitude and longitude coordinates below and changing the location name:

In [ ]:
lat, lon = 39.9526, -75.1652
location_name = "Philly"
filter_year = "2020"

We’ll use the coordinates and the year to filter out any unwanted scenes:

In [ ]:
def keep_scene(scene: Scene) -> bool:
    contains_location = (
        float(scene["min_lat"]) < lat
        and float(scene["max_lat"]) > lat
        and float(scene["min_lon"]) < lon
        and float(scene["max_lon"]) > lon
    )
    is_correct_year = "{}-".format(filter_year) in scene["acquisitionDate"]
    return contains_location and is_correct_year


location_scenes = [scene for scene in scenes if keep_scene(scene)]

This should leave us with a much more manageable subset of scenes:

In [ ]:
len(location_scenes)

We’ll be working with a single scene through the next few sections, so let’s use the first scene in our list:

In [ ]:
scene = location_scenes[0]
scene

#Read metadata from the MTL file
Landsat 8 metadata is contained in an MTL file that is alongside of the download_url file specified in the scene data. Let’s read the MTL file for the first scene and see what it looks like.

First we define a function that reads a file based on the download_url location - we’ll be using this a lot to get file URLs related to a scene:

In [ ]:
def get_asset_url(scene: Scene, suffix: str) -> str:
    product_id = scene["productId"]
    download_url = scene["download_url"]
    asset_filename = "{}_{}".format(product_id, suffix)
    return download_url.replace("index.html", asset_filename)

We can then use function to get the MTL file for our scene. Notice we use pystac.STAC_IO.read_text - this is the method that PySTAC uses to read text as it crawls a STAC. It can read from the local filesystem or HTTP/HTTPS by default. Also, it can be extended to read from other sources such as cloud providers - see the documentation here. For now we’ll use it directly as an easy way to read a text file from an HTTPS source:

In [ ]:
stac_io = pystac.StacIO.default()
mtl_url = get_asset_url(scene, "MTL.txt")
print(stac_io.read_text(mtl_url))

The MTL file contains metadata in a text format that’s a bit hard to use as-is; we can parse things out to a dict for easier access:

In [ ]:
def get_metadata(url: str) -> Dict[str, Any]:
    """Convert Landsat MTL file to dictionary of metadata values"""
    mtl = {}
    mtl_text = stac_io.read_text(url)
    for line in mtl_text.split("\n"):
        meta = line.replace('"', "").strip().split("=")
        if len(meta) > 1:
            key = meta[0].strip()
            item = meta[1].strip()
            if key != "GROUP" and key != "END_GROUP":
                mtl[key] = item
    return mtl

In [ ]:
metadata = get_metadata(mtl_url)
metadata

# Create a STAC Item from a scene
Now that we have metadata for the scene let’s use it to create a STAC Item.

We can use the help method to see the signature of the __init__ method on pystac.Item. You can also call help directly on pystac.Item for broader documentation, or check the API docs for Item here.

In [ ]:
help(pystac.Item.__init__

We can see we’ll need at least an id, geometry, bbox, and datetime. Properties is required, but can be an empty dictionary that we fill out on the Item once it’s created.

#Item id
For the Item’s id, we’ll use the scene ID. We’ll chop off the last 5 characters as they are repeated for each ID and so aren’t necessary:

In [ ]:
def get_item_id(metadata: Dict[str, Any]) -> str:
    return cast(str, metadata["LANDSAT_SCENE_ID"][:-5])

In [ ]:
item_id = get_item_id(metadata)
item_id

# Item datetime
Here we parse the datetime of the Item from two metadata fields that describe the date and time the scene was captured:

In [ ]:
from dateutil.parser import parse


def get_datetime(metadata: Dict[str, Any]) -> datetime:
    return parse("%sT%s" % (metadata["DATE_ACQUIRED"], metadata["SCENE_CENTER_TIME"]))

In [ ]:
item_datetime = get_datetime(metadata)
item_datetime

# Item bbox
Here we read in the bounding box information from the scene and transform it into the format of the Item’s bbox property:

In [ ]:
def get_bbox(metadata: Dict[str, Any]) -> List[float]:
    coords = [
        [
            [
                float(metadata["CORNER_UL_LON_PRODUCT"]),
                float(metadata["CORNER_UL_LAT_PRODUCT"]),
            ],
            [
                float(metadata["CORNER_UR_LON_PRODUCT"]),
                float(metadata["CORNER_UR_LAT_PRODUCT"]),
            ],
            [
                float(metadata["CORNER_LR_LON_PRODUCT"]),
                float(metadata["CORNER_LR_LAT_PRODUCT"]),
            ],
            [
                float(metadata["CORNER_LL_LON_PRODUCT"]),
                float(metadata["CORNER_LL_LAT_PRODUCT"]),
            ],
            [
                float(metadata["CORNER_UL_LON_PRODUCT"]),
                float(metadata["CORNER_UL_LAT_PRODUCT"]),
            ],
        ]
    ]
    lats = [c[1] for c in coords[0]]
    lons = [c[0] for c in coords[0]]
    return [min(lons), min(lats), max(lons), max(lats)]

In [ ]:
item_bbox = get_bbox(metadata)
item_bbox

# Item geometry
Getting the geometry of the scene is a little more tricky. The bounding box will be a axis-aligned rectangle of the area the scene occupies, but will not represent the true footprint of the image - Landsat 8 scenes are “tilted” according the the coordinate reference system, so there will be areas in the corner where no image data exists. When constructing a STAC Item it’s best if you have the Item geometry represent the true footprint of the assets.

To get the footprint of the scene we’ll read in another metadata file that lives alongside the MTL - the ANG.txt file. This function uses the ANG file and the bbox to construct the GeoJSON polygon that represents the footprint of the scene:

In [ ]:
def get_geometry(scene: Scene, bbox: List[float]) -> Dict[str, Any]:
    url = get_asset_url(scene, "ANG.txt")
    sz = []
    coords = []
    ang_text = stac_io.read_text(url)
    for line in ang_text.split("\n"):
        if "BAND01_NUM_L1T_LINES" in line or "BAND01_NUM_L1T_SAMPS" in line:
            sz.append(float(line.split("=")[1]))
        if (
            "BAND01_L1T_IMAGE_CORNER_LINES" in line
            or "BAND01_L1T_IMAGE_CORNER_SAMPS" in line
        ):
            coords.append(
                [float(l) for l in line.split("=")[1].strip().strip("()").split(",")]
            )
        if len(coords) == 2:
            break
    dlon = bbox[2] - bbox[0]
    dlat = bbox[3] - bbox[1]
    lons = [c / sz[1] * dlon + bbox[0] for c in coords[1]]
    lats = [((sz[0] - c) / sz[0]) * dlat + bbox[1] for c in coords[0]]
    coordinates = [
        [
            [lons[0], lats[0]],
            [lons[1], lats[1]],
            [lons[2], lats[2]],
            [lons[3], lats[3]],
            [lons[0], lats[0]],
        ]
    ]

    return {"type": "Polygon", "coordinates": coordinates}

In [ ]:
item_geometry = get_geometry(scene, item_bbox)
item_geometry

This would be a good time to check our work - we can print out the GeoJSON and use geojson.io to check and make sure we’re using scenes that overlap our location. If this footprint is somewhere unexpected in the world, make sure the Lat/Long coordinates are correct and in the right order!

In [ ]:
import json

print(json.dumps(item_geometry, indent=2))

# Create the item
Now that we have the required attributes for an Item we can create it:

In [ ]:
item = pystac.Item(
    id=item_id,
    datetime=item_datetime,
    geometry=item_geometry,
    bbox=item_bbox,
    properties={},
)

PySTAC has a validate method on STAC objects, which you can use to make sure you’re constructing things correctly. If there’s an issue the following line will throw an exception:



In [ ]:
item.validate()

# Add Ground Sample Distance to common metadata
We’ll add the Ground Sample Distance that is defined as part of the Item Common Metadata. We define this on the Item level as 30 meters, which is the GSD for most of the bands of Landsat 8. However, there are some bands that have a different resolution; we will account for this by setting the GSD explicitly for each of those bands below.

In [ ]:
item.common_metadata.gsd = 30.0

# Adding the EO extension
STAC has a rich set of extensions that allow STAC objects to encode information that is not part of the core spec but is used widely and standardized. An example of this is the eo extension, which encapsulates data that that represents a snapshot of the earth for a single date and time.

We can enable the eo extension for this item by using the ext property that exists on all STAC objects:

In [ ]:
eo_ext = EOExtension.ext(item, add_if_missing=True)

# Add cloud cover
Here we add cloud cover from the metadata as part of the eo extension.

In [ ]:
def get_cloud_cover(metadata: Dict[str, Any]) -> float:
    return float(metadata["CLOUD_COVER"])

In [ ]:
eo_ext.cloud_cover = get_cloud_cover(metadata)
eo_ext.cloud_cover

# Adding assets
STAC Items contain a list of Assets, which are a list of files that relate to the Item. In our case we’ll be cataloging each file related to the scene, including the Landsat 8 band files as well as the metadata files associated with the scene.

Here we define a dictionary that describes the band assets. We use the eo extension’s Band class to encapsulate information about the band each file represents, and also specify the Ground Sample Distance of each band:

In [ ]:
from pystac.extensions.eo import Band


class BandInfo(TypedDict):
    band: Band
    gsd: float


landsat_band_info: Dict[str, BandInfo] = {
    "B1": {
        "band": Band.create(
            name="B1",
            common_name="coastal",
            center_wavelength=0.48,
            full_width_half_max=0.02,
        ),
        "gsd": 30.0,
    },
    "B2": {
        "band": Band.create(
            name="B2",
            common_name="blue",
            center_wavelength=0.44,
            full_width_half_max=0.06,
        ),
        "gsd": 30.0,
    },
    "B3": {
        "band": Band.create(
            name="B3",
            common_name="green",
            center_wavelength=0.56,
            full_width_half_max=0.06,
        ),
        "gsd": 30.0,
    },
    "B4": {
        "band": Band.create(
            name="B4",
            common_name="red",
            center_wavelength=0.65,
            full_width_half_max=0.04,
        ),
        "gsd": 30.0,
    },
    "B5": {
        "band": Band.create(
            name="B5",
            common_name="nir",
            center_wavelength=0.86,
            full_width_half_max=0.03,
        ),
        "gsd": 30.0,
    },
    "B6": {
        "band": Band.create(
            name="B6",
            common_name="swir16",
            center_wavelength=1.6,
            full_width_half_max=0.08,
        ),
        "gsd": 30.0,
    },
    "B7": {
        "band": Band.create(
            name="B7",
            common_name="swir22",
            center_wavelength=2.2,
            full_width_half_max=0.2,
        ),
        "gsd": 30.0,
    },
    "B8": {
        "band": Band.create(
            name="B8",
            common_name="pan",
            center_wavelength=0.59,
            full_width_half_max=0.18,
        ),
        "gsd": 15.0,
    },
    "B9": {
        "band": Band.create(
            name="B9",
            common_name="cirrus",
            center_wavelength=1.37,
            full_width_half_max=0.02,
        ),
        "gsd": 30.0,
    },
    "B10": {
        "band": Band.create(
            name="B10",
            common_name="lwir11",
            center_wavelength=10.9,
            full_width_half_max=0.8,
        ),
        "gsd": 100.0,
    },
    "B11": {
        "band": Band.create(
            name="B11",
            common_name="lwir12",
            center_wavelength=12.0,
            full_width_half_max=1.0,
        ),
        "gsd": 100.0,
    },
}

There are also other non-band assets associated with a scene, and we specify the Asset’s URL and media type here, along with the key we will refer to each asset by:

In [ ]:
def get_other_assets(scene: Scene) -> Dict[str, Dict[str, Any]]:
    return {
        "thumbnail": {
            "href": get_asset_url(scene, "thumb_large.jpg"),
            "media_type": pystac.MediaType.JPEG,
        },
        "index": {
            "href": get_asset_url(scene, "index.html"),
            "media_type": "application/html",
        },
        "ANG": {"href": get_asset_url(scene, "ANG.txt"), "media_type": "text/plain"},
        "MTL": {"href": get_asset_url(scene, "MTL.txt"), "media_type": "text/plain"},
        "BQA": {
            "href": get_asset_url(scene, "BQA.TIF"),
            "media_type": pystac.MediaType.GEOTIFF,
        },
    }

With this information we can now define a method that adds all the relevant assets for a scene and add them to an item:

In [ ]:
def add_assets(item: pystac.Item, scene: Scene) -> None:
    # Add bands
    for band_id, band_info in landsat_band_info.items():
        band_url = get_asset_url(scene, "{}.TIF".format(band_id))
        asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
        bands = [band_info["band"]]
        asset_eo_ext = EOExtension.ext(asset)
        asset_eo_ext.bands = bands
        item.add_asset(band_id, asset)

        # If this asset has a different GSD than the item, set it on the asset
        if band_info["gsd"] != item.common_metadata.gsd:
            asset.common_metadata.gsd = band_info["gsd"]

    # Add other assets
    for asset_id, asset_info in get_other_assets(scene).items():
        item.add_asset(
            asset_id,
            pystac.Asset(href=asset_info["href"], media_type=asset_info["media_type"]),
        )

In [ ]:
add_assets(item, scene)

The logic for the Assets is such that if the gsd of the Asset is different from the Item’s GSD (30 meters), the Asset’s GSD will be specified directly on the Asset. We can see this by comparing the dict encoding of the Assets for band 10 and band 3:

In [ ]:
item.assets["B10"].to_dict()

In [ ]:
item.assets["B3"].to_dict()

Here we see the thumbnail asset, which does not include the band information for the eo extension as it does not represent any of the Item’s bands:

In [ ]:
item.assets["thumbnail"].to_dict()

# Add projection information
We can specify the EPSG code for the scene as part of the projection extension. The below method figures out the correct UTM Zone EPSG code based on the center latitude of the scene:

In [ ]:
def get_epsg(metadata: Dict[str, Any], min_lat: float, max_lat: float) -> Optional[int]:
    if "UTM_ZONE" in metadata:
        center_lat = (min_lat + max_lat) / 2.0
        return int(("326" if center_lat > 0 else "327") + metadata["UTM_ZONE"])
    else:
        return None

In [ ]:
proj_ext = ProjectionExtension.ext(item, add_if_missing=True)
assert item.bbox is not None
proj_ext.epsg = get_epsg(metadata, item.bbox[1], item.bbox[3])
proj_ext.epsg

# Add view geometry information
The View Geometry extension specifies information related to angles of sensors and other radiance angles that affect the view of resulting data. The Landsat 8 metadata specifies two of these parameters, so we add them to our Item:

In [ ]:
def get_view_info(metadata: Dict[str, Any]) -> Dict[str, float]:
    return {
        "sun_azimuth": float(metadata["SUN_AZIMUTH"]),
        "sun_elevation": float(metadata["SUN_ELEVATION"]),
    }

In [ ]:
view_ext = ViewExtension.ext(item, add_if_missing=True)
view_info = get_view_info(metadata)
view_ext.sun_azimuth = view_info["sun_azimuth"]
view_ext.sun_elevation = view_info["sun_elevation"]
item.properties

Now that we’ve added all the metadata to the item, let’s check the validator to make sure we’ve specified everything correctly. The validation logic will take into account the new extensions that have been enabled and validate against the proper schemas for those extensions.

In [ ]:
item.validate()

# Building the Collection
Now that we know how to build an item for a scene, let’s build the collection that will contain all the Items.

If we look at the __init__ method for pystac.Collection, we can see what properties are required:

In [ ]:
help(pystac.Collection.__init__)

# Collection id
We’ll use the location name we defined above in the ID for our Collection:

In [ ]:
collection_id = "{}-landsat-8".format(location_name)
collection_id

# Collection title
Here we set a simple title for our collection.

In [ ]:
collection_title = "2020 Landsat 8 images over {}".format(location_name)
collection_title

# Collection description
Here we give a brief description of the Collection. If this were a real Collection that were being published, I’d recommend putting a much more detailed description to ensure anyone using your STAC knows what they are working with!

Notice we are using Markdown to write the description. The description field can be Markdown to help tools that render information about STAC to display the information in a more readable way.

In [ ]:
collection_description = """### {} Landsat 8

A collection of Landsat 8 scenes around {} in 2020.
""".format(
    location_name, location_name
)
print(collection_description)

# Collection extent
A Collection specifies the spatial and temporal extent of all the item it contains. Since Landsat 8 spans the globe, we’ll simply put a global extent here. We’ll also specify an open-ended time interval that starts with the first datetime for scenes hosted by AWS.

Towards the end of the notebook, we’ll use a method to easily scope this down to cover the times and space the Items occupy once we’ve added all the items.

In [ ]:
from datetime import datetime

spatial_extent = pystac.SpatialExtent([[-180.0, -90.0, 180.0, 90.0]])
temporal_extent = pystac.TemporalExtent([[datetime(2013, 6, 1), None]])
collection_extent = pystac.Extent(spatial_extent, temporal_extent)

In [ ]:
collection = pystac.Collection(
    id=collection_id,
    title=collection_title,
    description=collection_description,
    extent=collection_extent,
)

We can now look at our Collection as a dict to check our values.

In [ ]:
collection.to_dict()

# Set the license
Notice the license above is proprietary. This is the default in PySTAC if no license is specified; however Landsat 8 is certainly not proprietary (thankfully!), so let’s change the license to the correct SPDX string for public domain data:

In [ ]:
collection_license = "PDDL-1.0"

# Set the providers
A collection will specify the providers of the data, including what role they have played. We can set our provider information by instantiating pystac.Provider objects:

In [ ]:
collection.providers = [
    pystac.Provider(
        name="USGS",
        roles=[pystac.ProviderRole.PRODUCER],
        url="https://landsat.usgs.gov/",
    ),
    pystac.Provider(
        name="Planet Labs",
        roles=[pystac.ProviderRole.PROCESSOR],
        url="https://github.com/landsat-pds/landsat_ingestor",
    ),
    pystac.Provider(
        name="AWS", roles=[pystac.ProviderRole.HOST], url="https://landsatonaws.com/"
    ),
]

# Create items for each scene
We created an Item for a single scene above. This method consolidates that logic into a single method that can construct an Item from a scene, so we can create an Item for every scene in our subset:

In [ ]:
def item_from_scene(scene: Scene) -> pystac.Item:
    mtl_url = get_asset_url(scene, "MTL.txt")
    metadata = get_metadata(mtl_url)

    bbox = get_bbox(metadata)
    item = pystac.Item(
        id=get_item_id(metadata),
        datetime=get_datetime(metadata),
        geometry=get_geometry(scene, bbox),
        bbox=bbox,
        properties={},
    )

    item.common_metadata.gsd = 30.0

    item_eo_ext = EOExtension.ext(item, add_if_missing=True)
    item_eo_ext.cloud_cover = get_cloud_cover(metadata)

    add_assets(item, scene)

    item_proj_ext = ProjectionExtension.ext(item, add_if_missing=True)
    assert item.bbox is not None
    item_proj_ext.epsg = get_epsg(metadata, item.bbox[1], item.bbox[3])

    item_view_ext = ViewExtension.ext(item, add_if_missing=True)
    view_info = get_view_info(metadata)
    item_view_ext.sun_azimuth = view_info["sun_azimuth"]
    item_view_ext.sun_elevation = view_info["sun_elevation"]

    item.validate()

    return item

Here we create an item per scene and add it to our collection. Since this is reading multiple metadata files per scene from the internet, it may take a little bit to run:

In [ ]:
for scene in location_scenes:
    item = item_from_scene(scene)
    collection.add_item(item)

## Reset collection extent based on items
Now that we’ve added all the item we can use the update_extent_from_items method on the Collection to set the extent based on the contained items:

In [ ]:
collection.update_extent_from_items()
collection.extent.to_dict()

# Set the HREFs for everything in the catalog
We’ve been building up our Collection and Items in memory. This has been convenient as it allows us not to think about file paths as we construct our Catalog. However, a STAC is not valid without any HREFs!

We can use the normalize_hrefs method to set all the HREFs in the entire STAC based on a root directory. This will use the STAC Best Practices recommendations for STAC file layout for each Catalog, Collection and Item in the STAC.

Here we use that method and set the root directory to a subdirectory of our user’s home directory:

In [ ]:
from pathlib import Path

root_path = str(Path.home() / "{}-landsat-stac".format(location_name))

collection.normalize_hrefs(root_path)

Now that we have all the Collection’s data set and HREFs in place we can validate the entire STAC using validate_all, which recursively crawls through a catalog and validates every STAC object in the catalog:

In [ ]:
collection.validate_all()

# Write the catalog locally
Now that we have our complete, validated STAC in memory, let’s write it out. This is as simple as calling save on the Collection. We need to specify the type of catalog in order to property write out links - these types are described again in the STAC Best Practices documentation.

We’ll use the “self contained” type, which uses relative paths and does not specify absolute “self” links to any object. This makes the catalog more portable, as it remains valid even if you copy the STAC to new locations.

In [ ]:
collection.save(pystac.CatalogType.SELF_CONTAINED)

Now that we’ve written our STAC out we probably want to view it. We can use the describe method to print out a simple representation of the catalog:

In [ ]:
collection.describe()

We can also use the to_dict method on individual STAC objects in order to see the data, as we’ve been doing in the tutorial:

In [ ]:
collection.to_dict()

However, if we want to browse our STAC more interactively, we can use the stac-browser tool to read our local STAC.

We can use this simple Python server (copied from this gist) to serve our our directory at port 5555:

In [ ]:
import os
from http.server import HTTPServer, SimpleHTTPRequestHandler

os.chdir(root_path)


class CORSRequestHandler(SimpleHTTPRequestHandler):
    def end_headers(self) -> None:
        self.send_header("Access-Control-Allow-Origin", "*")
        self.send_header("Access-Control-Allow-Methods", "GET")
        self.send_header("Cache-Control", "no-store, no-cache, must-revalidate")
        return super(CORSRequestHandler, self).end_headers()


with HTTPServer(("localhost", 5555), CORSRequestHandler) as httpd:
    httpd.serve_forever()

Now you can follow the stac-browser instructions for starting a stac-browser instance and point it at http://localhost:5555/collection.json to serve out the STAC!

To quit the server, use the Kernel -> Interrupt menu option.